<a href="https://colab.research.google.com/github/inthecng/SmartThingsPublic/blob/master/naver_land.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#데이터 크롤링

import requests
import pandas as pd

# API 기본 URL
url = "https://m.land.naver.com/cluster/ajax/articleList"

# 요청 헤더
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# 페이지 크기와 초기 페이지 번호 설정
page_size = 20  # 한 페이지당 항목 수
page_number = 1  # 시작 페이지

all_data = []

while True:
    # URL에 페이지와 페이지 크기 추가
    params = {
        "rletTpCd": "DDDGG",
        "tradTpCd": "B2",
        "z": 14,
        "lat": 37.0535034,
        "lon": 127.0258068,
        "btm": 37.0237004,
        "lft": 126.9792436,
        "top": 37.0832946,
        "rgt": 127.0723699,
        "showR0": "",
        "page": page_number,  # 현재 페이지
        "size": page_size  # 페이지 크기
    }

    response = requests.get(url, headers=headers, params=params, verify=False)

    if response.status_code == 200:
        data = response.json()

        # 데이터가 없으면 종료
        if not data.get('body'):
            break

        all_data.extend(data['body'])

        # 다음 페이지로 이동
        page_number += 1
    else:
        print(f"API 호출 실패: {response.status_code}")
        break

# 데이터를 DataFrame으로 변환
df_org = pd.DataFrame(all_data)
df = df_org.copy()


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'm.land.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'm.land.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'm.land.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.p

In [12]:
import gspread
import pandas as pd
from google.oauth2.service_account import Credentials

# 인증 범위 설정
scopes = ["https://www.googleapis.com/auth/spreadsheets"]

# 서비스 계정 인증서 로드
credentials = Credentials.from_service_account_file(
    'naverland-446111-584ea20bf298.json', scopes=scopes)

# gspread 클라이언트 생성
gc = gspread.authorize(credentials)

# 스프레드시트 URL로 열기
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1Eid96Vuvj-2FLDfVLxV98QUzAo0GbuItxqZv0UbxJT4/edit?usp=sharing'
spreadsheet = gc.open_by_url(spreadsheet_url)

# 특정 워크시트 선택
worksheet = spreadsheet.worksheet('column_code')

# 워크시트의 모든 데이터를 가져와 DataFrame으로 변환
data = worksheet.get_all_records()
code_df = pd.DataFrame(data)

# code_df의 'Code'와 '구분2' 컬럼을 딕셔너리로 변환
mapping_dict = dict(zip(code_df['Code'], code_df['구분2']))

# df라는 기존 DataFrame이 있다고 가정하고, 존재하는 컬럼에 대해서만 변경하기 위해 rename 메서드 사용
df = df.rename(columns=mapping_dict)

# 결과 출력 (선택 사항)
print(df)


         atclNo    cortarNo  단지명 atclStatCd rletTpCd uprRletTpCd    매물유형  \
0    2461860210  4122032000  다가구         R0      C03         C03  단독/다가구   
1    2501235181  4122012800  다가구         R0      C03         C03  단독/다가구   
2    2501231015  4122012800  다가구         R0      C03         C03  단독/다가구   
3    2501247468  4122012800  다가구         R0      C03         C03  단독/다가구   
4    2501254718  4122033000   단독         R0      C03         C03  단독/다가구   
..          ...         ...  ...        ...      ...         ...     ...   
597  2462177069  4122012800  다가구         R0      C03         C03  단독/다가구   
598  2462053511  4122032000  다가구         R0      C03         C03  단독/다가구   
599  2461653079  4122010100  다가구         R0      C03         C03  단독/다가구   
600  2461628592  4122010100  다가구         R0      C03         C03  단독/다가구   
601  2461172308  4122010900  다가구         R0      C03         C03  단독/다가구   

    tradTpCd 거래구분 vrfcTpCd  ...  \
0         B2   월세     S_VR  ...   
1         B2   월세